In [1]:
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import datetime as dt
import pandas as pd
import numpy as np


import holoviews as hv
import hvplot.xarray
from holoviews import opts
from holoviews import streams
hv.extension('bokeh')
import panel as pn
import hvplot.pandas
import holoviews.plotting.bokeh



plt.close('all')

In [49]:
# Loading in dataset: (duplicate cell for multiple datasets, and change variable name)

path = 'H:\\Summer24-25\\' ##### enter file path here ---> double slashes or change direction eg. \ --> \\ or / #####
name = 'o3_Amon_CESM2-WACCM_ssp126*.nc' ##### enter file name here --> use * if need multiple files/versions per dataset #####
ds = xr.open_mfdataset(path+name, parallel=True)

if ds.plev.attrs['units'] == 'Pa':
    ds['plev'] = ds.plev / 100
    ds.plev.attrs['axis'] = 'Z'
    ds.plev.attrs['positive'] = 'down'
    ds.plev.attrs['standard_name'] = 'air_pressure'
    ds.plev.attrs['title'] = 'pressure'
    ds.plev.attrs['type'] = 'double'
    ds.plev.attrs['units'] = 'hPa'

ds.plev

<xarray.DataArray 'plev' (plev: 19)> Size: 152B
array([1000.,  925.,  850.,  700.,  600.,  500.,  400.,  300.,  250.,  200.,
        150.,  100.,   70.,   50.,   30.,   20.,   10.,    5.,    1.])
Coordinates:
  * plev     (plev) float64 152B 1e+03 925.0 850.0 700.0 ... 20.0 10.0 5.0 1.0
Attributes:
    axis:           Z
    positive:       down
    standard_name:  air_pressure
    title:          pressure
    type:           double
    units:          hPa

# HVPLOTS

In [32]:
# Use isel (index select) to plot graph using index of coordinates
plt1 = ds.o3.isel(time=10, plev=8).hvplot(geo=True).opts(tools=["hover"], default_tools=[]) # plot at first time and pressure level indexes

plt2 = ds.o3.isel(time=10, plev=7).hvplot(geo=True
).opts(tools=["hover"], default_tools=[])
# plt2
plot = (plt1 + plt2).opts(merge_tools=True)
# hvplot.save(plot, 'ff.auto')
hvplot.show(plot)

Launching server at http://localhost:58535


In [34]:
hv.output(widget_location='bottom')
data = ds.o3.mean(dim='plev').isel(time=slice(0,36,1))
# norm = mpl.colors.Normalize(vmin=(10**-8), vmax=(5*10**-7))
x = data.hvplot(
    'lon', 'lat', projection=ccrs.Orthographic(0,-90), global_extent=False, project=True, 
    cmap='turbo', coastline=True, size=500, clim=(1.1*10**-6, 2.2*10**-6)
).opts(default_tools=[])
hvplot.save(x, 'mean.auto')

In [5]:
# ds.o3.sel(plev=20000).sort_values('month').hvplot.quadmesh(x='lat', y='lon', alpha=0.2)
ds.o3.sel(plev=200)[::2,::2].isel(time=slice(6,25,6)).hvplot(col='time', colorbar=False, clim=(10**-8, 5*10**-7))

:GridSpace   [time]
   :Image   [lon,lat]   (Mole Fraction of O3)

In [14]:
path = hv.Path([])
freehand = streams.FreehandDraw(source=path, num_objects=6,
                                styles={'line_color':'yellow'})

path.opts(
    opts.Path(active_tools=['freehand_draw'], height=1000, line_width=20, width=2000))
hvplot.show(path)

Launching server at http://localhost:50836


In [36]:
data = ds.o3.sel(plev=200).isel(time=slice(0,12*4,1))[2::,2::]
# norm = mpl.colors.Normalize(vmin=(10**-8), vmax=(5*10**-7))
x = data.hvplot(
    'lon', 'lat', global_extent=False, project=True, geo=True, # projection=ccrs.Orthographic(0,-90),
    cmap='turbo', clim=(10**-8, 5*10**-7), coastline=True, tiles=True, width=1000, height=1000
).opts(tools=['hover'], default_tools=[])

hvplot.show(x)

Launching server at http://localhost:58554


In [23]:
path = hv.Path([])
data = ds.o3.isel(time=slice(0,24), plev=slice(6,11))
freehand = streams.FreehandDraw(source=path, num_objects=6)
path.opts(opts.Path(active_tools=['freehand_draw'], height=500, line_width=20, width=1000))
graph = data.hvplot('lon','lat', projection=ccrs.EqualEarth(), project=True, coastline=True, size= 600)
path*graph
hvplot.save(path*graph, 'hhh.html')

In [3]:
### Not mine, example from website
def sine(frequency=1.0, amplitude=1.0, function='sin'):
    xs = np.arange(200)/200*20.0
    ys = amplitude*getattr(np, function)(frequency*xs)
    return pd.DataFrame(dict(y=ys), index=xs).hvplot(height=250, responsive=True)

dmap = hv.DynamicMap(sine, kdims=['frequency', 'amplitude', 'function']).redim.range(
    frequency=(0.1, 10), amplitude=(1, 10)).redim.values(function=['sin', 'cos', 'tan'])

dmap_panel = pn.panel(dmap, height=400, sizing_mode="stretch_width")
dmap_panel

pn.panel(dmap, center=True, widget_layout=pn.Row, widget_location='top_left')
hv_panel = pn.pane.HoloViews(dmap, widgets={
    'amplitude': pn.widgets.LiteralInput(value=1., type=(float, int)),
    'function': pn.widgets.RadioButtonGroup,
    'frequency': {'value': 5}
}).layout

hv_panel

Row(sizing_mode='stretch_width')
    [0] HoloViews(DynamicMap, height=250, sizing_mode='stretch_width', widgets={'amplitude': LiteralInput...})
    [1] WidgetBox(align=('end', 'start'))
        [0] FloatSlider(end=10, name='frequency', start=0.1, value=5)
        [1] LiteralInput(name='amplitude', type=(<class 'float'>, ..., value=1.0)
        [2] RadioButtonGroup(name='function', options=[np.str_('sin'), ...], value=np.str_('sin'))

# Matplotlib

In [3]:
%matplotlib qt

In [53]:
## adapted from https://matplotlib.org/stable/gallery/widgets/slider_demo.html and https://matplotlib.org/stable/gallery/widgets/slider_snap_demo.html

import matplotlib.pyplot as plt
import numpy as np

from matplotlib.widgets import Button, Slider

init_plev = 200
init_time = 0
fig = plt.figure()
ax = fig.add_subplot(projection=ccrs.Orthographic(central_latitude=-90))
ax.coastlines()
ax.set_global()
date = dt.strptime(str(ds.time[init_time].values), '%Y-%m-%d %H:%M:%S')
graph = ds.o3.isel(time=init_time).sel(plev=init_plev).plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree())
ax.set_title('Date: %s' %date.strftime("%Y %B"))

fig.subplots_adjust(left=0.2, bottom=0.2)

# horizontal slider --> time
axtime = fig.add_axes([0.25, 0.1, 0.65, 0.03])
time_slider = Slider(ax=axtime, label='Time (Month)', valmin=0, valmax=24, valstep=1, valinit=init_time,)

# vertical slider -> plev
axplev = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
plev_slider = Slider(ax=axplev, label='Pressure Level (hPa)', valmin=70, valmax=400, 
                     valstep=ds.plev, valinit=init_plev, orientation="vertical",)

def update(val):
    graph.set_array(ds.o3.isel(time=time_slider.val).sel(plev=plev_slider.val))
    date = dt.strptime(str(ds.time[time_slider.val].values), '%Y-%m-%d %H:%M:%S')
    ax.set_title('Date: %s' %date.strftime("%Y %B"))

# register the update function with each slider
time_slider.on_changed(update)
plev_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    time_slider.reset()
    plev_slider.reset()
    
button.on_clicked(reset)
plt.show()

In [6]:
%matplotlib module://matplotlib_inline.backend_inline

In [90]:
## from matplotlib docs -> https://matplotlib.org/stable/gallery/event_handling/ginput_manual_clabel_sgskip.html

import time

import matplotlib.pyplot as plt
import numpy as np


def tellme(s):
    plt.title(s, fontsize=16)
    plt.draw()

plt.figure()
plt.xlim(0, 1)
plt.ylim(0, 1)

tellme('You will define a triangle, click to begin')

plt.waitforbuttonpress()

while True:
    pts = []
    while len(pts) < 3:
        tellme('Select 3 corners with mouse')
        pts = np.asarray(plt.ginput(3, timeout=10, ))
        if len(pts) < 3:
            tellme('Too few points, starting over')
            time.sleep(1)  # Wait a second
    ph = plt.fill(pts[:, 0], pts[:, 1], 'r', lw=2)

    tellme('Happy? Key click for yes, mouse click for no')

    if plt.waitforbuttonpress():
        break

    # Get rid of fill
    for p in ph:
        p.remove()

# Define a nice function of distance from individual pts
def f(x, y, pts):
    z = np.zeros_like(x)
    for p in pts:
        z = z + 1/(np.sqrt((x - p[0])**2 + (y - p[1])**2))
    return 1/z


X, Y = np.meshgrid(np.linspace(-1, 1, 51), np.linspace(-1, 1, 51))
Z = f(X, Y, pts)

CS = plt.contour(X, Y, Z, 20)

tellme('Use mouse to select contour label locations, middle button to finish')
CL = plt.clabel(CS, manual=True)


tellme('Now do a nested zoom, click to begin')
plt.waitforbuttonpress()

while True:
    tellme('Select two corners of zoom, middle mouse button to finish')
    pts = plt.ginput(2, timeout=-1)
    if len(pts) < 2:
        break
    (x0, y0), (x1, y1) = pts
    xmin, xmax = sorted([x0, x1])
    ymin, ymax = sorted([y0, y1])
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

tellme('All Done!')
plt.show()

Select label locations manually using first mouse button.
End manual selection with second mouse button.
